## データチェック用のnotebook

エラーが出たときにscpでファイルをローカルに転送してこのnotebookで細かくチェックする

In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
from edinet_xbrl.edinet_xbrl_parser import EdinetXbrlParser
from bs4 import BeautifulSoup
import jaconv
import csv
import re
import sys
import os
from lib import myfunc

In [2]:
## init parser
parser = EdinetXbrlParser()

In [3]:
# どっちがどっちか要確認
# 単体 or 連結のkeyの辞書
fs_dict = {"non-consolidated": "jpcrp_cor:NotesTaxEffectAccountingFinancialStatementsTextBlock"
    , "consolidated": "jpcrp_cor:NotesTaxEffectAccountingConsolidatedFinancialStatementsTextBlock"}

context_ref = "CurrentYearDuration"

In [10]:
### チェック用のファイルと使用するキー
# ファイル
xbrl_file_path = "./data/jpcrp030000-asr-001_E04221-000_2019-03-31_01_2019-06-26.xbrl"
# キー
usekey = fs_dict["non-consolidated"]

In [11]:
# 対象ファイルをパース
edinet_xbrl_object = parser.parse_file(xbrl_file_path)

In [12]:
# 辞書のキーでカラム名を変える
if usekey == "consolidated":
    colname_tmp = "連結会計年度"
else:
    colname_tmp = "事業年度"

In [13]:
# テーブルの確認
table = myfunc.get_table(edinet_xbrl_object, usekey, context_ref)

# tableがない場合はスキップ
if table is None:
    # ToDo: loggingでlog出力したい
    print("There is no table.")

In [14]:
df = myfunc.table_to_pd(table)

In [18]:
df.head()

,account,前事業年度(2018年3月31日),前事業年度(2018年3月31日)_unit,,当事業年度(2019年3月31日),当事業年度(2019年3月31日)_unit
0,繰延税金資産,,,,,
1,貸倒引当金,"2,200",千円,,"2,107",千円
2,賞与引当金,"25,133",,,"24,001",
3,退職給付引当金,"2,413",,,"2,795",
4,役員退職慰労引当金,"11,608",,,"13,487",


In [18]:
# dfが6列の場合は単位のカラムの補完と値の数値変換を行う
if len(df.columns) == 6:
    df.loc[:, df.columns.str.contains("_unit")] = df.loc[:, df.columns.str.contains("_unit")].apply(myfunc.fill_unit)
    # 値のカラム
    collist_val = [i for i in df.columns[df.columns.str.contains(colname_tmp)] if re.search("^.*(?<!_unit)$", i)]
    df.loc[:, collist_val] = df.loc[:, collist_val].applymap(myfunc.get_value)

# dfが4列の場合は単位の分離を行う
if len(df.columns) == 4:
    # colname_tmpが含まれるカラムを取得
    colname_tmp_list = [i for i in df.columns if re.search(colname_tmp, i)]

    for col in colname_tmp_list:
        df = myfunc.sep_unit(df, col)